In [ ]:
from math import pi

import matplotlib.pyplot as plt

from ct import load_bitmap

# filename = "Kropka.jpg"
# filename = "Shepp_logan.jpg"
# filename = "SADDLE_PE-large.JPG"
filename = "CT_ScoutView-large.jpg"

scans_count = 360

alfa_step = pi / scans_count

detectors_count = 360

detectors_angular_aperture = 270.0 / 180.0 * pi

has_to_use_sinogram_filter = False
has_to_save_intermediate_steps = False
has_to_show_intermediate_steps = False

image = load_bitmap(filename)
plt.imshow(image)
image.close()

In [ ]:
from ct import simulateComputedTomographyScan


states = simulateComputedTomographyScan(
    filename,
    scans_count,
    alfa_step,
    detectors_count,
    detectors_angular_aperture,
    has_to_use_sinogram_filter,
    has_to_save_intermediate_steps,
    has_to_show_intermediate_steps,
)

In [ ]:
import ipywidgets

from ct import showStateAfterScan


ipywidgets.interact(
    showStateAfterScan,
    states=ipywidgets.fixed(states),
    scan=ipywidgets.IntSlider(
        min=1 if states[-1] != 1 else scans_count, max=scans_count
    ),
)

In [ ]:
import pydicom
from datetime import datetime, date, time

from ct import create_dicom_from_image, PatientInfo

image_data = states[3]
dicom_filename = filename.split(".")[0] + ".dcm"
patient_info = PatientInfo(
    "Jan", "Kowalski", identifier="123456", birth_date=date(1983, 2, 1)
)
study_datetime = datetime(2025, 4, 7, 8, 31, 42)
create_dicom_from_image(
    image_data,
    dicom_filename,
    patient_info,
    study_datetime,
    series_description="Przykladowy opis badania",
    private_comment="U pacjenta nie zauwazono nieprawidlowosci",
)

ds = pydicom.dcmread(dicom_filename)
print("Nazwisko i imię pacjenta:", str(ds.PatientName).replace("^", " "))
print("Data urodzenia pacjenta:", date.fromisoformat(ds.PatientBirthDate).isoformat())
print("Data badania:", date.fromisoformat(ds.StudyDate).isoformat())
print("Godzina badania:", time.fromisoformat(ds.StudyTime).isoformat())
print("Opis:", ds.SeriesDescription)
print("Komentarz:", ds.get_private_item(0x000B, 0x01, "Comments").value)

In [ ]:
import numpy as np


image = load_bitmap(filename)
original = np.asarray(image, dtype=np.float64).mean(axis=-1)
image.close()

In [ ]:
from ct import simulateComputedTomographyScan, mse

# I. liczba detektorów zmienia się od 90 do 720 z krokiem 90

mse_detectors = []
for detectors_count_r in range(90, 720 + 1, 90):
    states = simulateComputedTomographyScan(
        filename,
        scans_count,
        alfa_step,
        detectors_count_r,
        detectors_angular_aperture,
        has_to_use_sinogram_filter,
        has_to_save_intermediate_steps,
        has_to_show_intermediate_steps,
    )
    mse_detectors.append((detectors_count_r, mse(original, states[3])))

print("MSE zależne od liczby detektorów\n", mse_detectors)
plt.scatter(*zip(*mse_detectors))
plt.title("MSE by detectors count")
plt.xlabel("Detectors count")
plt.ylabel("Mean squared error")
plt.show()

In [ ]:
from ct import simulateComputedTomographyScan, mse

# II. liczba skanów zmienia się od 90 do 720 z krokiem 90

mse_scans = []
for scans_count_r in range(90, 720 + 1, 90):
    states = simulateComputedTomographyScan(
        filename,
        scans_count_r,
        pi / scans_count_r,
        detectors_count,
        detectors_angular_aperture,
        has_to_use_sinogram_filter,
        has_to_save_intermediate_steps,
        has_to_show_intermediate_steps,
    )
    mse_scans.append((scans_count_r, mse(original, states[3])))

print("MSE zależne od liczby skanów\n", mse_scans)
plt.scatter(*zip(*mse_scans))
plt.title("MSE by scans count")
plt.xlabel("Scans count")
plt.ylabel("Mean squared error")
plt.show()

In [ ]:
from ct import simulateComputedTomographyScan, mse

# III. rozpiętość wachlarza zmienia się od 45 do 270 stopni z krokiem 45 stopni.

mse_aperture = []
for detectors_angular_aperture_r in range(45, 270 + 1, 45):
    states = simulateComputedTomographyScan(
        filename,
        scans_count,
        alfa_step,
        detectors_count,
        detectors_angular_aperture_r * pi / 180,
        has_to_use_sinogram_filter,
        has_to_save_intermediate_steps,
        has_to_show_intermediate_steps,
    )
    mse_aperture.append((detectors_angular_aperture_r, mse(original, states[3])))

print("MSE zależne od apertury\n", mse_aperture)
plt.scatter(*zip(*mse_aperture))
plt.title("MSE by aperture")
plt.xlabel("Aperture (degrees)")
plt.ylabel("Mean squared error")
plt.show()

In [ ]:
states_without_filter = simulateComputedTomographyScan(
    filename,
    scans_count,
    alfa_step,
    detectors_count,
    detectors_angular_aperture,
    False,
    has_to_save_intermediate_steps,
    has_to_show_intermediate_steps,
)

states_with_filter = simulateComputedTomographyScan(
    filename,
    scans_count,
    alfa_step,
    detectors_count,
    detectors_angular_aperture,
    True,
    has_to_save_intermediate_steps,
    has_to_show_intermediate_steps,
)

mse_without_filter = mse(original, states_without_filter[3])
mse_with_filter = mse(original, states_with_filter[3])
print("With filter applied:", mse_with_filter)
print("Without filter:", mse_without_filter)

values = [mse_with_filter, mse_without_filter]
bars = plt.bar(["with filter applied", "without filter"], values)

for bar in bars:
    yval = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        yval,
        round(yval, 2),
        ha="center",
        va="bottom",
        fontsize=12,
    )
plt.ylim(0, max(values) * 1.2)

plt.ylabel("Mean squared error")
plt.title("MSE by applying filter")
plt.show()